In [ ]:
import urllib.request
import re
from bs4 import BeautifulSoup as soup
import pandas as pd
import os
import requests
import multiprocessing
import time

from multiprocessing import Pool
from joblib import Parallel, delayed

In [ ]:
my_url = 'https://www.adb.org/projects'
file_name='ADB_2'

#create a document to save downloaded pdf
try:
    os.mkdir(file_name)
except:
    pass

path=r'C:\Users\yzhhu\Desktop\G.A.I_summer\bank grabber\{}'.format(file_name)
web='https://www.adb.org'
hh='?page='
page_range=list(range(507))

#You will need this step to create a public list that each processor can manipulate
manager = multiprocessing.Manager()
projectlist = manager.list()
status_list=manager.list()
status_fail=manager.list()

In [ ]:
def save_pdf_include(url,name,status):
    opener=urllib.request.build_opener()
    opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
    urllib.request.install_opener(opener)
    os.chdir(path)
    try:
        os.mkdir(path+'/'+status)
    except:
        pass
    os.chdir(path+'/'+status)
    urllib.request.urlretrieve(url,name+'.pdf')
    print('save {} successfully'.format(name))

In [ ]:
#Get all urls
def get_url (page):
    linklist=[]
    projectlist=[]
    for i in page:
        linklist.append(my_url+hh+str(i))
    return linklist


# get project with page number

In [ ]:
linklist=get_url(page_range)
projectlist

In [ ]:
#Get project urls
def get_project (link):
    global projectlist
    r=requests.get(link)
    page_soup = soup(r.content, "html.parser")
    links = [a['href'] for a in page_soup.find_all('a', href=True)]
    for k in links:
        if ('/projects/' in k) and ('/main' in k):
            projectlist.append(k)

    print('finished {}'.format(link))

# get project link


In [ ]:
#Use multiprocessing to repeatedly do the same function(Here is the formula)

#Returns how many processors are in your computer
num_cores=multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(get_project)(j) for j in linklist)
#We have the project list now

In [ ]:
#To get all the project names and urls
name_list=list(map(lambda x:x.split('projects/')[1].split('/main')[0],projectlist))
project_final_list=list(map(lambda x: web+x+'#project-pds',projectlist))

In [ ]:
len(project_final_list)

# downloading....

In [ ]:
#Check the 'status' of each project and seperatly download them into different folders
def get_everything (link,name):
    #Call the public list you have created in the function you will put into multiprocessing
    global status_fail
    r=requests.get(link)
    ini_soup = soup(r.content, "html.parser")
    try:
        status=ini_soup.find_all('span',{'class':['project-status Proposed']})[0].string.split(' ')[1]
    except:
        pass
    try:
        status=ini_soup.find_all('span',{'class':['project-status Dropped / Terminated']})[0].string.split(' ')[1]
    except:
        pass
    try:
        status=ini_soup.find_all('span',{'class':['project-status Active']})[0].string.split(' ')[1]
    except:
        pass
    try:
        status=ini_soup.find_all('span',{'class':['project-status Approved']})[0].string.split(' ')[1]
    except:
        pass
    try:
        status=ini_soup.find_all('span',{'class':['project-status Closed']})[0].string.split(' ')[1]
    except:
        pass
    try:
        status=ini_soup.find_all('span',{'class':['project-status Archived']})[0].string.split(' ')[1]
    except:
        pass
    try:
        save_pdf_include(link, name, status)
        print('successfully append {}'.format(link))
      
    #For those project pdfs downloaded unsuccessfully we put them into another list
    except:
        status_fail.append(link)
        print('fking messed up with {}'.format(link))
    time.sleep(1)

In [ ]:
#Time count
start=time.time()
#Returns how many processors are in your computer
num_cores=multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(get_everything)(j,p) for j,p in zip(project_final_list,name_list))
print('------time used: {} ------------'.format(time.time()-start))